In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

input_ = pd.read_excel('./3.model.xlsx',
                       header=0,
                       dtype=str).fillna('')

length = len(input_)

print('总数量：' + str(length))
print()

# = = = = = = = = = = = = = = =

import requests
import json

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(length):
    work.put_nowait(i)

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        i = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for _ in range(101):
            try:
                jsn = '{"jsn":{"tab":"catalog","make":"' + input_.loc[i, 'Make'] + '","year":"' + input_.loc[i, 'Year'] + '","model":"' + input_.loc[i, 'Model'] + '","nodetype":"model"}}'

                payload = {'func': 'navnode_fetch',
                           'payload': jsn,
                           'api_json_request': '1'}

                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=payload,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text

                resp = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                html = resp['html_fill_sections']['navchildren[]']
                
                soup = BeautifulSoup(html, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_engine = [engine.strip() for engine in html.xpath('//td[@class="nlabel"]/a/text()')]

                # = = = = = = = = = = = = = = =
                
                list_url = ['https://www.rockauto.com'+url.strip() for url in html.xpath('//td[@class="niconspace ncollapsedicon"]/a/@href')]
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
                
            except:
                time.sleep(0.3)
                continue

        if crawler_status == 'ok':
            df_temp = pd.DataFrame({'Tab_Label': input_.loc[i, 'Tab_Label'],
                                    'No.': int(input_.loc[i, 'No.']),
                                    'Make': input_.loc[i, 'Make'],
                                    'Year': input_.loc[i, 'Year'],
                                    'Model': input_.loc[i, 'Model'],
                                    'Engine': list_engine,
                                    'Car_Code': [url.split(',')[-1] for url in list_url],
                                    'Url': list_url})

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
        else:
            df_temp = pd.DataFrame([{'Tab_Label': input_.loc[i, 'Tab_Label'],
                                     'No.': int(input_.loc[i, 'No.']),
                                     'Make': input_.loc[i, 'Make'],
                                     'Year': input_.loc[i, 'Year'],
                                     'Model': input_.loc[i, 'Model'],
                                     'Url': input_.loc[i, 'Url']}])
    
            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        print(input_.loc[i, 'Make'] + ' - ' + input_.loc[i, 'Year'] + ' - ' + input_.loc[i, 'Model'] +  '  <->  [' + crawler_status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['No.', 'Year', 'Model', 'Engine'], ascending=[True, False, True, True])
output_correct.to_excel('./4.engine-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.', 'Year', 'Model'], ascending=[True, False, True])
    output_error.to_excel('./engine_error.xlsx', index=False)
    print()
    print('爬虫存在error')
print()
print('爬虫结束')

总数量：34219

ABARTH - 1969 - SIMCA  <->  [ok] - 剩余数量：34209 - 当前时间：17:16:20
ABARTH - 1969 - 1000  <->  [ok] - 剩余数量：34208 - 当前时间：17:16:21
ABARTH - 1968 - 2000  <->  [ok] - 剩余数量：34207 - 当前时间：17:16:21
ABARTH - 1967 - 1000  <->  [ok] - 剩余数量：34206 - 当前时间：17:16:21
ABARTH - 1967 - 2000  <->  [ok] - 剩余数量：34205 - 当前时间：17:16:21
ABARTH - 1968 - SIMCA  <->  [ok] - 剩余数量：34204 - 当前时间：17:16:22
ABARTH - 1968 - 1300  <->  [ok] - 剩余数量：34203 - 当前时间：17:16:22
ABARTH - 1966 - 1000  <->  [ok] - 剩余数量：34202 - 当前时间：17:16:22
ABARTH - 1966 - 1600  <->  [ok] - 剩余数量：34201 - 当前时间：17:16:22
ABARTH - 1966 - 1150  <->  [ok] - 剩余数量：34200 - 当前时间：17:16:23
ABARTH - 1967 - 1300  <->  [ok] - 剩余数量：34199 - 当前时间：17:16:23
ABARTH - 1965 - 1000  <->  [ok] - 剩余数量：34198 - 当前时间：17:16:24
ABARTH - 1965 - 1600  <->  [ok] - 剩余数量：34197 - 当前时间：17:16:24
ABARTH - 1966 - 850  <->  [ok] - 剩余数量：34196 - 当前时间：17:16:24
ABARTH - 1965 - 850  <->  [ok] - 剩余数量：34195 - 当前时间：17:16:24
ABARTH - 1965 - SIMCA  <->  [ok] - 剩余数量：34194 - 当前时间：17:16:25
ABARTH - 196

ACURA - 2018 - ILX  <->  [ok] - 剩余数量：34071 - 当前时间：17:16:57
ACURA - 2016 - ILX  <->  [ok] - 剩余数量：34070 - 当前时间：17:16:58
ACURA - 2015 - MDX  <->  [ok] - 剩余数量：34069 - 当前时间：17:16:58
ACURA - 2015 - RDX  <->  [ok] - 剩余数量：34068 - 当前时间：17:16:58
ACURA - 2016 - RLX  <->  [ok] - 剩余数量：34067 - 当前时间：17:16:58
ACURA - 2016 - TLX  <->  [ok] - 剩余数量：34066 - 当前时间：17:16:58
ACURA - 2016 - RDX  <->  [ok] - 剩余数量：34065 - 当前时间：17:16:58
ACURA - 2015 - TLX  <->  [ok] - 剩余数量：34064 - 当前时间：17:16:58
ACURA - 2014 - RLX  <->  [ok] - 剩余数量：34063 - 当前时间：17:16:59
ACURA - 2015 - RLX  <->  [ok] - 剩余数量：34062 - 当前时间：17:16:59
ACURA - 2014 - TL  <->  [ok] - 剩余数量：34061 - 当前时间：17:16:59
ACURA - 2015 - ILX  <->  [ok] - 剩余数量：34060 - 当前时间：17:16:59
ACURA - 2014 - TSX  <->  [ok] - 剩余数量：34059 - 当前时间：17:16:59
ACURA - 2014 - RDX  <->  [ok] - 剩余数量：34058 - 当前时间：17:16:59
ACURA - 2014 - MDX  <->  [ok] - 剩余数量：34057 - 当前时间：17:17:00
ACURA - 2013 - MDX  <->  [ok] - 剩余数量：34056 - 当前时间：17:17:00
ACURA - 2013 - TSX  <->  [ok] - 剩余数量：34055 - 当前时间：17:17:0

ACURA - 2001 - INTEGRA  <->  [ok] - 剩余数量：33932 - 当前时间：17:17:30
ACURA - 1992 - LEGEND  <->  [ok] - 剩余数量：33931 - 当前时间：17:17:30
ACURA - 1992 - INTEGRA  <->  [ok] - 剩余数量：33930 - 当前时间：17:17:31
ACURA - 1992 - NSX  <->  [ok] - 剩余数量：33929 - 当前时间：17:17:31
ACURA - 1995 - INTEGRA  <->  [ok] - 剩余数量：33928 - 当前时间：17:17:31
ACURA - 1991 - INTEGRA  <->  [ok] - 剩余数量：33927 - 当前时间：17:17:31
ACURA - 1995 - TL  <->  [ok] - 剩余数量：33926 - 当前时间：17:17:31
ACURA - 1991 - LEGEND  <->  [ok] - 剩余数量：33925 - 当前时间：17:17:32
ACURA - 1989 - INTEGRA  <->  [ok] - 剩余数量：33924 - 当前时间：17:17:33
ACURA - 1991 - NSX  <->  [ok] - 剩余数量：33923 - 当前时间：17:17:33
ACURA - 1990 - INTEGRA  <->  [ok] - 剩余数量：33922 - 当前时间：17:17:33
ACURA - 1989 - LEGEND  <->  [ok] - 剩余数量：33921 - 当前时间：17:17:33
ACURA - 1998 - RL  <->  [ok] - 剩余数量：33920 - 当前时间：17:17:33
ACURA - 1990 - LEGEND  <->  [ok] - 剩余数量：33919 - 当前时间：17:17:33
ACURA - 1988 - LEGEND  <->  [ok] - 剩余数量：33918 - 当前时间：17:17:34
ACURA - 1987 - LEGEND  <->  [ok] - 剩余数量：33917 - 当前时间：17:17:34
ACURA - 1986 - I

ALFA ROMEO - 1974 - GT VELOCE  <->  [ok] - 剩余数量：33806 - 当前时间：17:18:04
ALFA ROMEO - 1974 - SPIDER  <->  [ok] - 剩余数量：33805 - 当前时间：17:18:04
ALFA ROMEO - 1988 - MILANO  <->  [ok] - 剩余数量：33804 - 当前时间：17:18:05
ALFA ROMEO - 1973 - BERLINA  <->  [ok] - 剩余数量：33803 - 当前时间：17:18:05
ALFA ROMEO - 1973 - MONTREAL  <->  [ok] - 剩余数量：33802 - 当前时间：17:18:05
ALFA ROMEO - 1973 - SPIDER  <->  [ok] - 剩余数量：33801 - 当前时间：17:18:06
ALFA ROMEO - 1972 - GT VELOCE  <->  [ok] - 剩余数量：33800 - 当前时间：17:18:06
ALFA ROMEO - 1972 - BERLINA  <->  [ok] - 剩余数量：33799 - 当前时间：17:18:07
ALFA ROMEO - 1973 - GT VELOCE  <->  [ok] - 剩余数量：33798 - 当前时间：17:18:07
ALFA ROMEO - 1972 - SPIDER  <->  [ok] - 剩余数量：33797 - 当前时间：17:18:08
ALFA ROMEO - 1971 - BERLINA  <->  [ok] - 剩余数量：33796 - 当前时间：17:18:09
ALFA ROMEO - 1971 - MONTREAL  <->  [ok] - 剩余数量：33795 - 当前时间：17:18:09
ALFA ROMEO - 1971 - SPIDER  <->  [ok] - 剩余数量：33794 - 当前时间：17:18:10
ALFA ROMEO - 1970 - BERLINA  <->  [ok] - 剩余数量：33793 - 当前时间：17:18:10
ALFA ROMEO - 1970 - GT VELOCE  <->  [ok] - 剩余

ALLARD - 1946 - K1  <->  [ok] - 剩余数量：33681 - 当前时间：17:18:52
ALLARD - 1950 - J2  <->  [ok] - 剩余数量：33680 - 当前时间：17:18:52
ALLSTATE - 1953 - A-230  <->  [ok] - 剩余数量：33679 - 当前时间：17:18:53
ALLSTATE - 1953 - A-240  <->  [ok] - 剩余数量：33678 - 当前时间：17:18:53
ALLSTATE - 1952 - A-230  <->  [ok] - 剩余数量：33677 - 当前时间：17:18:53
ALLARD - 1946 - M  <->  [ok] - 剩余数量：33676 - 当前时间：17:18:53
ALLSTATE - 1952 - A-240  <->  [ok] - 剩余数量：33675 - 当前时间：17:18:53
ALPINE - 1973 - A310  <->  [ok] - 剩余数量：33674 - 当前时间：17:18:54
ALPINE - 1974 - A310  <->  [ok] - 剩余数量：33673 - 当前时间：17:18:54
ALPINE - 1971 - A310  <->  [ok] - 剩余数量：33672 - 当前时间：17:18:55
ALPINE - 1972 - A310  <->  [ok] - 剩余数量：33671 - 当前时间：17:18:55
ALPINE - 1969 - A110  <->  [ok] - 剩余数量：33670 - 当前时间：17:18:55
ALPINE - 1968 - A110  <->  [ok] - 剩余数量：33669 - 当前时间：17:18:56
ALPINE - 1970 - A110  <->  [ok] - 剩余数量：33668 - 当前时间：17:18:56
ALPINE - 1967 - A110  <->  [ok] - 剩余数量：33667 - 当前时间：17:18:56
ALPINE - 1966 - A110  <->  [ok] - 剩余数量：33666 - 当前时间：17:18:56
ALPINE - 1965 - A11

...

VOLVO - 2002 - S80  <->  [ok] - 剩余数量：576 - 当前时间：20:55:24
VOLVO - 2002 - V70  <->  [ok] - 剩余数量：575 - 当前时间：20:55:25
VOLVO - 2002 - V40  <->  [ok] - 剩余数量：574 - 当前时间：20:55:25
VOLVO - 2001 - C70  <->  [ok] - 剩余数量：573 - 当前时间：20:55:25
VOLVO - 2001 - S40  <->  [ok] - 剩余数量：572 - 当前时间：20:55:26
VOLVO - 2001 - S60  <->  [ok] - 剩余数量：571 - 当前时间：20:55:26
VOLVO - 2004 - C70  <->  [ok] - 剩余数量：570 - 当前时间：20:55:27
VOLVO - 2003 - V40  <->  [ok] - 剩余数量：569 - 当前时间：20:55:27
VOLVO - 2001 - V40  <->  [ok] - 剩余数量：568 - 当前时间：20:55:27
VOLVO - 2006 - V70  <->  [ok] - 剩余数量：567 - 当前时间：20:55:28
VOLVO - 2001 - V70  <->  [ok] - 剩余数量：566 - 当前时间：20:55:28
VOLVO - 2004 - S40  <->  [ok] - 剩余数量：565 - 当前时间：20:55:28
VOLVO - 2000 - S70  <->  [ok] - 剩余数量：564 - 当前时间：20:55:28
VOLVO - 2000 - C70  <->  [ok] - 剩余数量：563 - 当前时间：20:55:29
VOLVO - 2000 - V70  <->  [ok] - 剩余数量：562 - 当前时间：20:55:29
VOLVO - 2000 - S40  <->  [ok] - 剩余数量：561 - 当前时间：20:55:29
VOLVO - 2000 - V40  <->  [ok] - 剩余数量：560 - 当前时间：20:55:29
VOLVO - 2000 - S80  <->  [ok] -

VOLVO - 1972 - 1800  <->  [ok] - 剩余数量：432 - 当前时间：20:56:28
VOLKSWAGEN - 1973 - SQUAREBACK  <->  [ok] - 剩余数量：431 - 当前时间：20:56:28
VOLVO - 1971 - 144  <->  [ok] - 剩余数量：430 - 当前时间：20:56:29
VOLVO - 1971 - 145  <->  [ok] - 剩余数量：429 - 当前时间：20:56:29
VOLVO - 1971 - 164  <->  [ok] - 剩余数量：428 - 当前时间：20:56:29
VOLVO - 1973 - 145  <->  [ok] - 剩余数量：427 - 当前时间：20:56:30
VOLVO - 1970 - 145  <->  [ok] - 剩余数量：426 - 当前时间：20:56:31
VOLVO - 1970 - 144  <->  [ok] - 剩余数量：425 - 当前时间：20:56:31
VOLVO - 1971 - 1800  <->  [ok] - 剩余数量：424 - 当前时间：20:56:35
VOLVO - 1969 - 142  <->  [ok] - 剩余数量：423 - 当前时间：20:56:36
VOLVO - 1972 - 144  <->  [ok] - 剩余数量：422 - 当前时间：20:56:36
VOLVO - 1978 - 244  <->  [ok] - 剩余数量：421 - 当前时间：20:56:36
VOLVO - 1969 - 145  <->  [ok] - 剩余数量：420 - 当前时间：20:56:37
VOLVO - 1970 - 142  <->  [ok] - 剩余数量：419 - 当前时间：20:56:37
VOLVO - 1969 - 144  <->  [ok] - 剩余数量：418 - 当前时间：20:56:37
VOLVO - 1968 - 122  <->  [ok] - 剩余数量：417 - 当前时间：20:56:38
VOLVO - 1969 - 1800  <->  [ok] - 剩余数量：416 - 当前时间：20:56:38
VOLVO - 1970 - 1

WILLYS - 1934 - MODEL 77  <->  [ok] - 剩余数量：295 - 当前时间：20:57:33
WILLYS - 1933 - MODEL 6-90A  <->  [ok] - 剩余数量：294 - 当前时间：20:57:34
WILLYS - 1933 - MODEL 8-88A  <->  [ok] - 剩余数量：293 - 当前时间：20:57:35
WILLYS - 1932 - KNIGHT MODEL 66D  <->  [ok] - 剩余数量：292 - 当前时间：20:57:35
WILLYS - 1933 - KNIGHT MODEL 66E  <->  [ok] - 剩余数量：291 - 当前时间：20:57:35
WILLYS - 1932 - MODEL 6-90  <->  [ok] - 剩余数量：290 - 当前时间：20:57:36
WILLYS - 1932 - KNIGHT MODEL 95 DELUXE  <->  [ok] - 剩余数量：289 - 当前时间：20:57:36
WILLYS - 1932 - MODEL 97  <->  [ok] - 剩余数量：288 - 当前时间：20:57:37
WILLYS - 1932 - MODEL 8-80D  <->  [ok] - 剩余数量：287 - 当前时间：20:57:38
WILLYS - 1932 - MODEL 8-88  <->  [ok] - 剩余数量：286 - 当前时间：20:57:38
WILLYS - 1931 - KNIGHT MODEL 66D  <->  [ok] - 剩余数量：285 - 当前时间：20:57:39
WILLYS - 1931 - KNIGHT MODEL 66B  <->  [ok] - 剩余数量：284 - 当前时间：20:57:39
WILLYS - 1939 - MODEL 38 TRUCK  <->  [ok] - 剩余数量：283 - 当前时间：20:57:39
WILLYS - 1932 - MODEL 98D  <->  [ok] - 剩余数量：282 - 当前时间：20:57:40
WILLYS - 1931 - MODEL 6-87  <->  [ok] - 剩余数量：281 - 当

WOLSELEY - 1961 - 1500  <->  [ok] - 剩余数量：171 - 当前时间：20:58:15
WOLSELEY - 1958 - 6/90  <->  [ok] - 剩余数量：170 - 当前时间：20:58:15
WOLSELEY - 1959 - 1500  <->  [ok] - 剩余数量：169 - 当前时间：20:58:15
WOLSELEY - 1958 - 15/60  <->  [ok] - 剩余数量：168 - 当前时间：20:58:17
WOLSELEY - 1957 - 6/90  <->  [ok] - 剩余数量：167 - 当前时间：20:58:17
WOLSELEY - 1957 - 4/44  <->  [ok] - 剩余数量：166 - 当前时间：20:58:17
WOLSELEY - 1956 - 15/50  <->  [ok] - 剩余数量：165 - 当前时间：20:58:17
WOLSELEY - 1956 - 4/44  <->  [ok] - 剩余数量：164 - 当前时间：20:58:19
WOLSELEY - 1955 - 15/50  <->  [ok] - 剩余数量：163 - 当前时间：20:58:19
WOLSELEY - 1956 - 6/90  <->  [ok] - 剩余数量：162 - 当前时间：20:58:19
WOLSELEY - 1955 - 4/44  <->  [ok] - 剩余数量：161 - 当前时间：20:58:20
WOLSELEY - 1954 - 4/44  <->  [ok] - 剩余数量：160 - 当前时间：20:58:20
WOLSELEY - 1954 - 6/80  <->  [ok] - 剩余数量：159 - 当前时间：20:58:21
WOLSELEY - 1955 - 6/90  <->  [ok] - 剩余数量：158 - 当前时间：20:58:22
WOLSELEY - 1953 - 4/44  <->  [ok] - 剩余数量：157 - 当前时间：20:58:22
WOLSELEY - 1957 - 15/50  <->  [ok] - 剩余数量：156 - 当前时间：20:58:22
WOLSELEY - 1963 - 6/

WORKHORSE - 2001 - FASTRACK FT1460  <->  [ok] - 剩余数量：38 - 当前时间：20:59:02
WORKHORSE - 2002 - P32  <->  [ok] - 剩余数量：37 - 当前时间：20:59:03
WORKHORSE - 2001 - P30  <->  [ok] - 剩余数量：36 - 当前时间：20:59:03
WORKHORSE - 2001 - FASTRACK FT1800  <->  [ok] - 剩余数量：35 - 当前时间：20:59:03
WORKHORSE - 2001 - P32  <->  [ok] - 剩余数量：34 - 当前时间：20:59:04
WORKHORSE - 2001 - FASTRACK FT1600  <->  [ok] - 剩余数量：33 - 当前时间：20:59:04
WORKHORSE - 2001 - P42  <->  [ok] - 剩余数量：32 - 当前时间：20:59:04
WORKHORSE - 2001 - W20  <->  [ok] - 剩余数量：31 - 当前时间：20:59:04
WORKHORSE - 2001 - W22  <->  [ok] - 剩余数量：30 - 当前时间：20:59:05
WORKHORSE - 2000 - FASTRACK FT1260  <->  [ok] - 剩余数量：29 - 当前时间：20:59:05
WORKHORSE - 2000 - FASTRACK FT1800  <->  [ok] - 剩余数量：28 - 当前时间：20:59:05
WORKHORSE - 2000 - FASTRACK FT1600  <->  [ok] - 剩余数量：27 - 当前时间：20:59:05
WORKHORSE - 2000 - P30  <->  [ok] - 剩余数量：26 - 当前时间：20:59:05
WORKHORSE - 2000 - P42  <->  [ok] - 剩余数量：25 - 当前时间：20:59:06
WORKHORSE - 1999 - P42  <->  [ok] - 剩余数量：24 - 当前时间：20:59:06
YELLOW CAB - 1929 - MODEL O-